In [1]:
from torchvision import transforms
from torch.utils.data import DataLoader

from utils.dataset import CustomDataset
from model.aotgan import InpaintGenerator

import utils

import torch
import os
import pandas as pd 
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import matplotlib.pyplot as plt
import cv2
import numpy as np 


In [2]:
ts_img_dir = '/mnt/HDD/octc/BACKUP/External_ds/data'
ts_mask_dir = '/mnt/HDD/octc/BACKUP/External_ds/mask_filtered'

test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

test_dataset = CustomDataset(
    image_dir = ts_img_dir,
    mask_dir = ts_mask_dir,
    transform= test_transform,
    mask_transform= test_transform,
    testing = True,
    mask_shuffle = False,
)
ts_batch = 16

test_loader = DataLoader(dataset = test_dataset, batch_size = ts_batch, shuffle = False)


In [3]:
load_path = '/mnt/HDD/oci_models/aotgan/OCI-GAN_v1_240506/model_79.pt'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
netG = InpaintGenerator().to(device)
G_check = torch.load(load_path)['netG_state_dict']
netG.load_state_dict(G_check)

<All keys matched successfully>

In [5]:
with torch.no_grad():

    netG.eval()
    for images, masks, image_paths in test_loader:
        images, masks = images.to(device), masks.to(device)
        ### inference
        pred_images = netG(images, masks)  # 3+1ch
        
        ## mask에서 0이 아닌 부분을 GT로 대체, 이때 마스크는 0~1사이의 값을 가짐 
        comp_images = images.clone()
        comp_images[masks.repeat(1,3,1,1) != 0] = pred_images[masks.repeat(1,3,1,1) != 0] #comp_image를 남겨두는 이유는 result를 확인하기 위함 
        utils.plotting(images, masks, comp_images)


torch.Size([16, 3, 512, 512]) torch.Size([16, 3, 512, 512])


RuntimeError: Given groups=1, weight of size [64, 4, 7, 7], expected input[16, 6, 518, 518] to have 4 channels, but got 6 channels instead